<a href="https://colab.research.google.com/github/EmKelsall/ADS2002---Catheter-Placement-Project/blob/main/ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/ADS2002-Project-CatheterPlacement

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1sDmXQTrvwQhCF8mWDh9phnwM9axaYjjb/ADS2002-Project-CatheterPlacement


In [ ]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from PIL import Image
import ast
import os
import cv2

In [ ]:
train = pd.read_csv('Data/train.csv')
train.head()

,StudyInstanceUID,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,Swan Ganz Catheter Present,PatientID
0,1.2.826.0.1.3680043.8.498.26697628953273228189...,0,0,0,0,0,0,1,0,0,0,0,ec89415d1
1,1.2.826.0.1.3680043.8.498.46302891597398758759...,0,0,1,0,0,1,0,0,0,1,0,bf4c6da3c
2,1.2.826.0.1.3680043.8.498.23819260719748494858...,0,0,0,0,0,0,0,0,1,0,0,3fc1c97e5
3,1.2.826.0.1.3680043.8.498.68286643202323212801...,0,0,0,0,0,0,0,1,0,0,0,c31019814
4,1.2.826.0.1.3680043.8.498.10050203009225938259...,0,0,0,0,0,0,0,0,0,1,0,207685cd1


In [ ]:
filenames = os.listdir("Data/train")
train_imgs = []
FileIDs = []
lst = []


desired_channels = 3

count = 0
for file in filenames:
    if count == 500:
        break

    # Making sure the images are the same pixel size
    img = Image.open(f"Data/train/{file}")
    img = img.resize((224, 224))
    img = np.array(img)

    # Ensure that the image has 3 channels (RGB)
    if img.shape[-1] != 3:
        img = np.stack((img,) * 3, axis=-1)  # Convert grayscale to RGB by duplicating channels

    train_imgs.append(img)  # Append the 4D image to the list

    # Save filename for later
    FileIDs.append(file)

    count += 1

# Convert the list of images to a numpy array
train_imgs = np.array(train_imgs)

In [ ]:
#extracting just the CVC labels
train_data = pd.read_csv("/content/drive/MyDrive/ADS2002-Project-CatheterPlacement/Data/train.csv")
CVC_train = train_data[["StudyInstanceUID","CVC - Abnormal", "CVC - Borderline", "CVC - Normal"]]
CVC_train.set_index('StudyInstanceUID', inplace=True)
CVC_train.head()

,CVC - Abnormal,CVC - Borderline,CVC - Normal
StudyInstanceUID,,,
1.2.826.0.1.3680043.8.498.26697628953273228189375557799582420561,0,0,0
1.2.826.0.1.3680043.8.498.46302891597398758759818628675365157729,0,0,1
1.2.826.0.1.3680043.8.498.23819260719748494858948050424870692577,0,1,0
1.2.826.0.1.3680043.8.498.68286643202323212801283518367144358744,1,0,0
1.2.826.0.1.3680043.8.498.10050203009225938259119000528814762175,0,0,1


In [ ]:
CVC_train['CVC - Present'] = np.where((CVC_train['CVC - Abnormal'] == 0) & (CVC_train['CVC - Normal'] == 0) & (CVC_train['CVC - Borderline'] == 0), 0, 1)

<ipython-input-11-8abb49231a47>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CVC_train['CVC - Present'] = np.where((CVC_train['CVC - Abnormal'] == 0) & (CVC_train['CVC - Normal'] == 0) & (CVC_train['CVC - Borderline'] == 0), 0, 1)


In [ ]:
CVC_train['CVC - Present'].value_counts()

1    29333
0      750
Name: CVC - Present, dtype: int64

In [ ]:
#cleaning up the labels
UID = []
for i in range(len(FileIDs)):
  UID.append(FileIDs[i].replace(".jpg",""))

UID[0:10]

['1.2.826.0.1.3680043.8.498.96373614405512283046314488555604647941',
 '1.2.826.0.1.3680043.8.498.96315500926641444675409331764690933307',
 '1.2.826.0.1.3680043.8.498.96450515721778915344219421534269786290',
 '1.2.826.0.1.3680043.8.498.96255059007178680952242469330314808585',
 '1.2.826.0.1.3680043.8.498.96311228469094629679821868712032248786',
 '1.2.826.0.1.3680043.8.498.96359554498564333142663951388648557789',
 '1.2.826.0.1.3680043.8.498.96447675290915183668080373345495953473',
 '1.2.826.0.1.3680043.8.498.96279880013483713441521686300695558821',
 '1.2.826.0.1.3680043.8.498.96349754766067875463134738379718157835',
 '1.2.826.0.1.3680043.8.498.96258089029863343818062710263042584654']

In [ ]:
train_labels = np.empty((0,3))
data = []

for ID in UID:
    data.append(CVC_train.loc[ID])

train_labels_df = pd.DataFrame(data)



train_labels_df = train_labels_df.reindex(UID)
train_labels = train_labels_df['CVC - Present'].values
# train_labels = train_labels_df.values
train_labels

array([1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0,

In [ ]:
from sklearn.model_selection import train_test_split

# Split your data into training and temporary data (combined validation and test)
train_imgs, temp_imgs, train_labels, temp_labels = train_test_split(train_imgs, train_labels, test_size=0.3, random_state=42)

# Split the temporary data into validation and test sets
val_imgs, test_imgs, val_labels, test_labels = train_test_split(temp_imgs, temp_labels, test_size=0.5, random_state=42)

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Data Preparation
# Load and preprocess your image data and labels here.

# Model Building
base_model = ResNet50(weights='imagenet', include_top=False)  # Load ResNet-50 without top layers

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)  # Add custom fully connected layers
predictions = Dense(1, activation='sigmoid')(x)  # Output layer with sigmoid activation for binary classification

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Model Training
model.fit(train_imgs, train_labels, epochs=5, batch_size=32, validation_data=(val_imgs, val_labels))

# Model Evaluation
test_loss, test_accuracy = model.evaluate(test_imgs, test_labels)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')


94765736/94765736 [==============================] - 0s 0us/step
Epoch 1/5
11/11 [==============================] - 196s 16s/step - loss: 0.6685 - accuracy: 0.8886 - val_loss: 213524.4375 - val_accuracy: 0.9733
Epoch 2/5
11/11 [==============================] - 175s 16s/step - loss: 0.1655 - accuracy: 0.9743 - val_loss: 9507922.0000 - val_accuracy: 0.9733
Epoch 3/5
11/11 [==============================] - 176s 16s/step - loss: 0.0995 - accuracy: 0.9743 - val_loss: 7123271.0000 - val_accuracy: 0.9733
Epoch 4/5
11/11 [==============================] - 178s 16s/step - loss: 0.0765 - accuracy: 0.9743 - val_loss: 698982.8125 - val_accuracy: 0.9733
Epoch 5/5
3/3 [==============================] - 8s 2s/step - loss: 41991.1055 - accuracy: 0.9867
Test Accuracy: 98.67%


In [ ]:
# New labels: CVC - Abnormal, CVC - Borderline, CVC - Normal
new_labels = CVC_train[["CVC - Abnormal", "CVC - Borderline", "CVC - Normal"]]

# Ensure labels are aligned with the images based on StudyInstanceUID
new_labels = new_labels.loc[UID]

# Load ResNet-50 without top layers
base_model = ResNet50(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)  # Output layer with softmax for multi-class classification

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Check if there are missing labels
missing_labels = train_imgs.shape[0] - new_labels.shape[0]

if missing_labels > 0:
    print(f"Warning: {missing_labels} image(s) have missing labels. Removing them.")
    train_imgs = train_imgs[:-missing_labels]
elif missing_labels < 0:
    print(f"Warning: {abs(missing_labels)} label(s) are missing corresponding images. Removing them.")
    new_labels = new_labels.iloc[:train_imgs.shape[0]]

# Split your data into training and temporary data (combined validation and test)
train_imgs, temp_imgs, train_labels, temp_labels = train_test_split(train_imgs, new_labels, test_size=0.3, random_state=42)

# Split the temporary data into validation and test sets
val_imgs, test_imgs, val_labels, test_labels = train_test_split(temp_imgs, temp_labels, test_size=0.5, random_state=42)

# Model Training
model.fit(train_imgs, train_labels, epochs=5, batch_size=32, validation_data=(val_imgs, val_labels))

# Model Evaluation
test_loss, test_accuracy = model.evaluate(test_imgs, test_labels)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')


Epoch 1/5
8/8 [==============================] - 136s 15s/step - loss: 3.6574 - accuracy: 0.4612 - val_loss: 691.2194 - val_accuracy: 0.6538
Epoch 2/5
8/8 [==============================] - 128s 16s/step - loss: 1.0894 - accuracy: 0.5551 - val_loss: 657.5157 - val_accuracy: 0.2692
Epoch 3/5
8/8 [==============================] - 125s 16s/step - loss: 1.2935 - accuracy: 0.5510 - val_loss: 3609.7656 - val_accuracy: 0.0769
Epoch 4/5
8/8 [==============================] - 125s 16s/step - loss: 1.6346 - accuracy: 0.5388 - val_loss: 1190.3544 - val_accuracy: 0.6538
Epoch 5/5
2/2 [==============================] - 6s 2s/step - loss: 301594624.0000 - accuracy: 0.6038
Test Accuracy: 60.38%
